In [71]:
from Bio import SeqIO
import re
from collections import Counter
import pandas as pd
import os.path
import panel as pn

In [76]:
PATH_TO_MIDORI_FOLDER = '../../../MIDORI'
PATH_TO_MIDORI_TABLE = '../../interim/MIDORI'
#df = pd.DataFrame(columns=['Species'])
def parser(taxa, gene):
    all_sp = []
    db = f'{PATH_TO_MIDORI_FOLDER}/{gene}.fasta'
    for entry in SeqIO.parse(db, 'fasta'):
        if taxa in entry.id:
            taxonomy = entry.id.split(';')
            all_sp.append(taxonomy[7])
    counted_sp = Counter(all_sp)
    
    if os.path.isfile(f'{PATH_TO_MIDORI_TABLE}/midori_sp_table.csv'):
        df = pd.read_csv(f'{PATH_TO_MIDORI_TABLE}/midori_sp_table.csv')
    else: 
        df = pd.DataFrame()
    sp_name = []
    gene_count = []
    for k,v in counted_sp.items():
        sp_name.append(k)
        gene_count.append(v)
    
    # CHECK IF IT'S EMPTY AND IF IT'S NOT, THEN MERGE BY SPECIES
    tmp_df = pd.DataFrame({'Species':sp_name, gene:gene_count})
    if df.empty:   
        df = df.append(tmp_df)
    else:
        df = pd.merge(df, tmp_df, how="outer")
    df = df.fillna(0)
    df.sort_values(by=['Species'], inplace=True)
    df.to_csv(f'{PATH_TO_MIDORI_TABLE}/midori_sp_table.csv', index=False)
    




In [77]:
genes = ['A6', 'A8', 'CO1', 'CO2', 'CO3', 'Cytb', 'ND1', 'ND2', 'ND3', 'ND4', 'ND4L', 'ND5', 'ND6']
for gene in genes:
    parser('Insecta', gene)

/tmp/ipykernel_11599/3690595901.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp_df)


In [90]:
df = pd.read_csv(f'{PATH_TO_MIDORI_TABLE}/midori_sp_table.csv')
print(len(df['Species']))

125214


In [89]:
def generate_html(dataframe: pd.DataFrame):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id="table")
    # construct the complete HTML with jQuery Data tables
    # You can disable paging or enable y scrolling on lines 20 and 21 respectively
    html = f"""
    <html>
    <header>
        <link href="https://cdn.datatables.net/1.11.5/css/jquery.dataTables.min.css" rel="stylesheet">
    </header>
    <body>
    {table_html}
    <script src="https://code.jquery.com/jquery-3.6.0.slim.min.js" integrity="sha256-u7e5khyithlIdTpu22PHhENmPcRdFiHRjhAuHcs05RI=" crossorigin="anonymous"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.11.5/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                // paging: false,    
                // scrollY: 400,
            }});
        }});
    </script>
    </body>
    </html>
    """
    # return the html
    return html

df = pd.read_csv(f'{PATH_TO_MIDORI_TABLE}/midori_sp_table.csv')
    # take only first 1000, otherwise it'll generate a large html file
df = df.iloc[:1000]
    # generate the HTML from the dataframe
html = generate_html(df)
    # write the HTML content to an HTML file
open("index.html", "w").write(html)

41342302